<img src='img/WhatsApp Image 2023-04-27 at 10.52.43.jpg' />

<img src='img/WhatsApp Image 2023-04-27 at 10.53.04.jpg' />

In [2]:
import findspark
findspark.init("/usr/local/spark")

from pyspark.sql import SparkSession 
import pyspark.sql.functions as F 
from pyspark.ml import Pipeline
from pyspark.sql.types import *

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
from delta import *

# Import Spark NLP
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline


builder = SparkSession.builder.appName("Sentiment Analysis - presentation") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.4.0")\
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .master("spark://namenode:7077")\
    .config("spark.executor.memory", "6g") \
    .config("spark.executor.cores", 4) 
    #.config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    

    
spark = configure_spark_with_delta_pip(builder).getOrCreate()


:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-02fd3d59-f346-48e8-b8fb-2e6ed8366c90;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 471ms :: artifacts dl 26ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   

In [3]:
# Prepare Dataframes
# Review data
spark.sparkContext.setJobDescription('load review dataset')
review_schema = StructType([StructField("review_id", StringType(), False),
      StructField("user_id", StringType(), False),
      StructField("business_id", StringType(), False),
      StructField("stars", StringType(), False), 
      StructField("useful", IntegerType(), False),
      StructField("funny", IntegerType(), False),
      StructField("cool", IntegerType(), False),
      StructField("text", StringType(), False),
      StructField("date", StringType(), False),])
# review_df = spark.read.csv("hdfs://namenode:9000/project_data/review.csv", sep = '|', header = False, schema = review_schema)
review_df = spark.read.csv("hdfs://namenode:9000/project_data/review_small_a.csv", sep = '|', header = False, schema = review_schema)
review_df.createOrReplaceTempView("reviews")
#review_df.persist()

#review_df.show()

# Business data
spark.sparkContext.setJobDescription('load review dataset')
business_schema = StructType([
    StructField("business_id", StringType(), False),
    StructField("name", StringType(), True),
    StructField("address", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("stars", DoubleType(), True),
    StructField("review_count", IntegerType(), True),
    StructField("is_open", IntegerType(), True),
    StructField("attributes", StringType(), True),
    StructField("categories", StringType(), True),
    StructField("hours", StringType(), True)
])
business_df = spark.read.csv("hdfs://namenode:9000/project_data/business.csv", sep = '|', header = False, schema = business_schema)
business_df.createOrReplaceTempView("business")
#business_df.persist()

## Test optimisation
### Combine and Filter

In [4]:
# Combine data on common restaurant id
spark.sparkContext.setJobGroup("combine datasets", "combine datasets")
reviews_with_category = spark.sql("SELECT r.review_id, b.business_id, r.text, r.date, b.categories, r.stars FROM reviews AS r LEFT JOIN business AS b ON b.business_id = r.business_id ")

# Filter for only restaurant reviews
spark.sparkContext.setJobGroup("filter reviews", "filter reviews")
restaurant_reviews = reviews_with_category.where(F.col('categories').contains("Restaurants"))
restaurant_reviews.show()

+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|           review_id|         business_id|                text|               date|          categories|stars|
+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|lr1xBBahWP2ZOqGco...|rv_1S0yg-YtulZ6gG...|Great coffee shop...|2020-06-08 02:07:44|Breakfast & Brunc...|  5.0|
|VqHhMjGJI_9bTcgl6...|8ikj9GaC2oEf_leis...|This was so incre...|2016-05-15 16:22:32|Diners, Restauran...|  5.0|
|h_UAfe8YoIpN5OGiH...|elSnD7Vuxnlg5_9gT...|decent interior f...|2017-01-10 16:20:32|Restaurants, Pizz...|  2.0|
|yYhliOOeTZXlV_ZWb...|kfpwO_cdVQdTh2_oM...|The American Sard...|2013-08-22 04:02:49|Restaurants, Bars...|  4.0|
|4okNZR_Z3whwioH3i...|BxfvdHqETU8jWYUjx...|I love shake shac...|2012-08-05 12:19:57|Food, Hot Dogs, B...|  4.0|
|0oCbH-K0tFa0Exbjl...|UakVMT3xrpbFB2pHd...|Well I finally ha...|2018-06-01 22:47:56|Bars, Nightlife, ...

In [4]:
restaurant_reviews.count()

4724468

### Filter and Combine
We expect this to be faster since the restaurant dataframe is first made smaller..

In [4]:
# Filter for only restaurant reviews
spark.sparkContext.setJobDescription('filter reviews')
business_df = business_df.where(F.col('categories').contains("Restaurants"))
business_df.createOrReplaceTempView("business")

# Combine data on common restaurant id
spark.sparkContext.setJobDescription('combine datasets')
restaurant_reviews = spark.sql("SELECT r.review_id, b.business_id, r.text, r.date, b.categories, r.stars FROM reviews AS r INNER JOIN business AS b ON b.business_id = r.business_id ")

restaurant_reviews.show()

+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|           review_id|         business_id|                text|               date|          categories|stars|
+--------------------+--------------------+--------------------+-------------------+--------------------+-----+
|lr1xBBahWP2ZOqGco...|rv_1S0yg-YtulZ6gG...|Great coffee shop...|2020-06-08 02:07:44|Breakfast & Brunc...|  5.0|
|VqHhMjGJI_9bTcgl6...|8ikj9GaC2oEf_leis...|This was so incre...|2016-05-15 16:22:32|Diners, Restauran...|  5.0|
|h_UAfe8YoIpN5OGiH...|elSnD7Vuxnlg5_9gT...|decent interior f...|2017-01-10 16:20:32|Restaurants, Pizz...|  2.0|
|yYhliOOeTZXlV_ZWb...|kfpwO_cdVQdTh2_oM...|The American Sard...|2013-08-22 04:02:49|Restaurants, Bars...|  4.0|
|4okNZR_Z3whwioH3i...|BxfvdHqETU8jWYUjx...|I love shake shac...|2012-08-05 12:19:57|Food, Hot Dogs, B...|  4.0|
|0oCbH-K0tFa0Exbjl...|UakVMT3xrpbFB2pHd...|Well I finally ha...|2018-06-01 22:47:56|Bars, Nightlife, ...

In [4]:
restaurant_reviews.count()

4724468

### MODEL and Training

In [5]:
spark.sparkContext.setJobDescription('pipeline builder')
# Build a pipeline
# Document Assembler
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

spark.sparkContext.setJobDescription('pipeline -tokenizer')
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")#\
    #.fit(restaurant_reviews) # apperantly works without this

spark.sparkContext.setJobDescription('pipeline -normalizer')
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normal")

spark.sparkContext.setJobDescription('pipeline -model')
vivekn = ViveknSentimentModel.pretrained() \
    .setInputCols(["document", "normal"]) \
    .setOutputCol("result_sentiment")

spark.sparkContext.setJobDescription('pipeline -model=finish')
finisher = Finisher() \
    .setInputCols(["result_sentiment"]) \
    .setOutputCols("final_sentiment")

spark.sparkContext.setJobDescription('pipeline -pipeline executor')
pipeline = Pipeline().setStages([documentAssembler, tokenizer, normalizer, vivekn, finisher])

spark.sparkContext.setJobDescription('pipeline -pipeline fit')
pipelineModel = pipeline.fit(restaurant_reviews)

spark.sparkContext.setJobDescription('pipeline -pipeline transform')
# Calculate sentiment for all restaurant reviews
result = pipelineModel.transform(restaurant_reviews)

spark.sparkContext.setJobDescription('pipeline -result check')
# Check accuracy


sentiment_vivekn download started this may take some time.
Approximate size to download 873.6 KB
[ | ]sentiment_vivekn download started this may take some time.
Approximate size to download 873.6 KB
[ / ]Download done! Loading the resource.
[OK!]


### finding Accuracy without persist 

In [ ]:
spark.sparkContext.setJobDescription('count right ones')
result = result.withColumn("right_prediction", 
                   F.when(((F.array_contains(F.col("final_sentiment"),"positive")) & (F.col("stars").isin(["5.0", "4.0", "3.0"]))) |
                        ((F.array_contains(F.col("final_sentiment"),"negative")) & (F.col("stars").isin(["3.0", "2.0", "1.0"]))), 
                        1).otherwise(0))

count_ones = result.agg(F.sum("right_prediction")).collect()[0][0]

#result gets dropped from the memory!!!

print(f"Prediction accuracy for sentiment: {count_ones/total_count}")
result.write.format("delta").mode("overwrite").save("/temp/sentiment_predicted_restaurant_reviews")


Prediction accuracy for sentiment: 0.64


<img src='img/withoutpersist.png' />

### finding accuracy with persist()

In [7]:
result.persist()

spark.sparkContext.setJobDescription('count right ones')
#### TO-DO --- try to optomise below line.
result = result.withColumn("right_prediction", 
                   F.when(((F.array_contains(F.col("final_sentiment"),"positive")) & (F.col("stars").isin(["5.0", "4.0", "3.0"]))) |
                        ((F.array_contains(F.col("final_sentiment"),"negative")) & (F.col("stars").isin(["3.0", "2.0", "1.0"]))), 
                        1).otherwise(0))
count_ones = result.agg(F.sum("right_prediction")).collect()[0][0]

total_count = result.count()
print(f"Prediction accuracy for sentiment: {count_ones/total_count}")

# result.write.format("delta").mode("overwrite").save("/temp/sentiment_predicted_restaurant_reviews")


Prediction accuracy for sentiment: 0.6666666666666666


<img src='img/WhatsApp Image 2023-04-27 at 09.45.30.jpg' />

In [6]:
result.count()

23

### upsert 


In [8]:
# delta_table_path = "/temp/sentiment_predicted_restaurant_reviews"
delta_table_path = "/temp/upsert_presentation_test1"
# Check if the Delta table exists
if DeltaTable.isDeltaTable(spark, delta_table_path):
    print("Updating delta")
    # If the Delta table exists, load it into a DataFrame
    deltaTableAnalyzedReviews = DeltaTable.forPath(spark, delta_table_path)
    deltaTableAnalyzedReviews.alias('old') \
        .merge(
        result.alias('updates'),
        'old.review_id = updates.review_id'
  ) \
  .whenMatchedUpdate(set =
    {
      "review_id": "updates.review_id",
      "business_id": "updates.business_id",
      "text": "updates.text",
      "date": "updates.date",
      "categories": "updates.categories",
      "stars":"updates.stars",
      "final_sentiment": "updates.final_sentiment",
      "right_prediction": "updates.right_prediction"
    }
  ) \
  .whenNotMatchedInsert(values =
    {
      "review_id": "updates.review_id",
      "business_id": "updates.business_id",
      "text": "updates.text",
      "date": "updates.date",
      "categories": "updates.categories",
      "stars":"updates.stars",
      "final_sentiment": "updates.final_sentiment",
      "right_prediction": "updates.right_prediction"
    }
  ) \
  .execute()
else:
    # If the Delta table does not exist, create it
    print("Creating delta")
    deltaTableAnalyzedReviews = result.write.format('delta').mode('overwrite').save(delta_table_path)
    deltaTableAnalyzedReviews = DeltaTable.forPath(spark, delta_table_path)
print("Finished!")

Updating delta


Finished!


In [9]:
deltaTableAnalyzedReviews = DeltaTable.forPath(spark, delta_table_path)
df = deltaTableAnalyzedReviews.toDF()

# Use the DataFrame as desired
print(df.count())
df.show()

29
+--------------------+--------------------+--------------------+-------------------+--------------------+-----+---------------+----------------+
|           review_id|         business_id|                text|               date|          categories|stars|final_sentiment|right_prediction|
+--------------------+--------------------+--------------------+-------------------+--------------------+-----+---------------+----------------+
|-KOxrHhTSayEk01XH...|CrP6JWXBmf_HyMnZJ...|Good place to eat...|2013-02-20 17:34:12|Restaurants, Burg...| null|     [positive]|               1|
|0oCbH-K0tFa0Exbjl...|UakVMT3xrpbFB2pHd...|Well I finally ha...|2018-06-01 22:47:56|Bars, Nightlife, ...|  5.0|     [positive]|               1|
|2ecMImprVq9UhB6Pe...|iUZEGx29miZObLd6_...|Wow! First off, I...|2009-11-20 16:54:17|Breakfast & Brunc...|  5.0|     [positive]|               1|
|4F0rrBlJ2GIbWcNNW...|UakVMT3xrpbFB2pHd...|Great breakfast b...|2018-12-24 18:49:00|Bars, Nightlife, ...| null|     [positive]|